# Payload 3D Orientation Animator

This notebook creates 3D animations showing the payload orientation (yaw/pitch/roll) over time by animating a rectangular prism.

## Setup and Library Installation

Install required libraries for 3D visualization and animation.

In [8]:
# Install required packages
import subprocess
import sys

packages = ['matplotlib', 'pandas', 'numpy']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("All packages installed successfully")

All packages installed successfully


## Import Libraries

Import necessary libraries for data processing and 3D animation.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pathlib import Path
from io import StringIO

print("Libraries imported successfully")

Libraries imported successfully


## Load Flight Data

Load and parse flight log data, splitting into separate flights.

In [10]:
def load_flight_data(filepath):
    """
    Load flight data from a txt file and split into separate flights.
    Returns a list of DataFrames, one for each flight.
    """
    flights = []
    current_flight = []
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    # Skip header and parse data
    header_line = None
    for i, line in enumerate(lines):
        line = line.strip()
        
        # Store header
        if 'millis' in line and header_line is None:
            header_line = line
            continue
        
        # Check for end of flight marker
        if '[---END OF FLIGHT---]' in line:
            if current_flight:
                data_str = header_line + '\n' + '\n'.join(current_flight)
                df = pd.read_csv(StringIO(data_str))
                flights.append(df)
                current_flight = []
            continue
        
        # Add data line to current flight
        if line and not line.startswith('['):
            current_flight.append(line)
    
    # Handle last flight if no end marker
    if current_flight:
        data_str = header_line + '\n' + '\n'.join(current_flight)
        df = pd.read_csv(StringIO(data_str))
        flights.append(df)
    
    return flights

# Load the flight data
log_file = "flight_log_11_20_25.txt"
flights = load_flight_data(f"data/{log_file}")
filename = Path(log_file).stem

# Create output directory
output_dir = Path("animations") / filename
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Loaded {len(flights)} flight(s) from {log_file}")
for i, flight in enumerate(flights, 1):
    print(f"  Flight {i}: {len(flight)} data points")

Loaded 3 flight(s) from flight_log_11_20_25.txt
  Flight 1: 29174 data points
  Flight 2: 74741 data points
  Flight 3: 57823 data points


## Define Rotation Functions

Functions to apply yaw, pitch, and roll rotations to 3D coordinates.

In [11]:
def rotation_matrix_x(angle):
    """Rotation matrix for roll (rotation around X-axis)."""
    angle = np.radians(angle)
    return np.array([
        [1, 0, 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)]
    ])

def rotation_matrix_y(angle):
    """Rotation matrix for pitch (rotation around Y-axis)."""
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle), 0, np.sin(angle)],
        [0, 1, 0],
        [-np.sin(angle), 0, np.cos(angle)]
    ])

def rotation_matrix_z(angle):
    """Rotation matrix for yaw (rotation around Z-axis)."""
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]
    ])

def apply_rotation(vertices, yaw, pitch, roll):
    """Apply yaw, pitch, roll rotations to vertices."""
    # Apply rotations in order: yaw -> pitch -> roll
    R_yaw = rotation_matrix_z(yaw)
    R_pitch = rotation_matrix_y(pitch)
    R_roll = rotation_matrix_x(roll)
    
    # Combined rotation matrix
    R = R_roll @ R_pitch @ R_yaw
    
    # Apply rotation to all vertices
    return np.dot(vertices, R.T)

print("Rotation functions defined")

Rotation functions defined


## Create Rectangle Geometry

Define the vertices and faces of the payload rectangle.

In [12]:
def create_rectangle_vertices(length=2, width=1, height=0.3):
    """
    Create vertices for a rectangular prism centered at origin.
    Default dimensions represent a typical payload shape.
    """
    l, w, h = length/2, width/2, height/2
    
    vertices = np.array([
        [-l, -w, -h],  # 0: bottom-back-left
        [l, -w, -h],   # 1: bottom-back-right
        [l, w, -h],    # 2: bottom-front-right
        [-l, w, -h],   # 3: bottom-front-left
        [-l, -w, h],   # 4: top-back-left
        [l, -w, h],    # 5: top-back-right
        [l, w, h],     # 6: top-front-right
        [-l, w, h]     # 7: top-front-left
    ])
    
    return vertices

def get_rectangle_faces(vertices):
    """Define the 6 faces of the rectangular prism."""
    faces = [
        [vertices[0], vertices[1], vertices[2], vertices[3]],  # Bottom
        [vertices[4], vertices[5], vertices[6], vertices[7]],  # Top
        [vertices[0], vertices[1], vertices[5], vertices[4]],  # Back
        [vertices[2], vertices[3], vertices[7], vertices[6]],  # Front
        [vertices[0], vertices[3], vertices[7], vertices[4]],  # Left
        [vertices[1], vertices[2], vertices[6], vertices[5]]   # Right
    ]
    
    return faces

print("Rectangle geometry functions defined")

Rectangle geometry functions defined


## Create Animation

Generate 3D animation showing payload orientation over time.

In [13]:
def create_orientation_animation(df, flight_num, filename, skip_frames=10):
    """
    Create 3D animation of payload orientation.
    
    Args:
        df: Flight data DataFrame
        flight_num: Flight number
        filename: Base filename for output
        skip_frames: Only animate every Nth frame to reduce file size
    """
    # Downsample data for animation
    df_sampled = df.iloc[::skip_frames].reset_index(drop=True)
    
    print(f"Creating animation for Flight {flight_num} ({len(df_sampled)} frames)...")
    
    # Create figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Set up the plot limits and labels
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([-2, 2])
    ax.set_xlabel('X', fontsize=10)
    ax.set_ylabel('Y', fontsize=10)
    ax.set_zlabel('Z', fontsize=10)
    
    # Create initial rectangle
    base_vertices = create_rectangle_vertices()
    
    # Initialize collection for the rectangle
    poly_collection = Poly3DCollection([], alpha=0.7, edgecolors='black', linewidths=1)
    ax.add_collection3d(poly_collection)
    
    # Text annotations
    time_text = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, fontsize=10)
    yaw_text = ax.text2D(0.02, 0.90, '', transform=ax.transAxes, fontsize=9)
    pitch_text = ax.text2D(0.02, 0.85, '', transform=ax.transAxes, fontsize=9)
    roll_text = ax.text2D(0.02, 0.80, '', transform=ax.transAxes, fontsize=9)
    altitude_text = ax.text2D(0.02, 0.75, '', transform=ax.transAxes, fontsize=9)
    state_text = ax.text2D(0.02, 0.70, '', transform=ax.transAxes, fontsize=9, fontweight='bold')
    
    def update(frame):
        """Update function for animation."""
        row = df_sampled.iloc[frame]
        
        # Get orientation
        yaw = row['yaw_deg']
        pitch = row['pitch_deg']
        roll = row['roll_deg']
        
        # Apply rotation to vertices
        rotated_vertices = apply_rotation(base_vertices, yaw, pitch, roll)
        
        # Get faces for the rotated rectangle
        faces = get_rectangle_faces(rotated_vertices)
        
        # Update polygon collection
        poly_collection.set_verts(faces)
        
        # Color based on state
        state_colors = {
            'GROUND': '#808080',
            'ASCENT': '#3498db',
            'DESCENT': '#e74c3c',
            'LANDED': '#95a5a6'
        }
        color = state_colors.get(row['state'], '#2ecc71')
        poly_collection.set_facecolor(color)
        
        # Update text
        time_seconds = row['millis'] / 1000
        time_text.set_text(f'Time: {time_seconds:.1f}s')
        yaw_text.set_text(f'Yaw: {yaw:.1f}°')
        pitch_text.set_text(f'Pitch: {pitch:.1f}°')
        roll_text.set_text(f'Roll: {roll:.1f}°')
        altitude_text.set_text(f'Altitude: {row["altitude_ft"]:.1f} ft')
        state_text.set_text(f'State: {row["state"]}')
        state_text.set_color(color)
        
        return poly_collection, time_text, yaw_text, pitch_text, roll_text, altitude_text, state_text
    
    # Create animation
    anim = FuncAnimation(
        fig, 
        update, 
        frames=len(df_sampled),
        interval=50,  # 50ms between frames = 20 fps
        blit=False
    )
    
    # Save animation
    output_path = output_dir / f"{filename}_orientation_f{flight_num}.gif"
    writer = PillowWriter(fps=20)
    anim.save(output_path, writer=writer)
    
    plt.close()
    print(f"Saved: {output_path}")

print("Animation function defined")

Animation function defined


## Generate Animations for All Flights

Create orientation animations for each flight in the log file.

In [14]:
# Generate animations (using skip_frames=20 to reduce file size)
for i, flight_df in enumerate(flights, 1):
    create_orientation_animation(flight_df, i, filename, skip_frames=20)

print(f"\nAll animations saved to: {output_dir}")

Creating animation for Flight 1 (1459 frames)...
Saved: animations/flight_log_11_20_25/flight_log_11_20_25_orientation_f1.gif
Creating animation for Flight 2 (3738 frames)...
Saved: animations/flight_log_11_20_25/flight_log_11_20_25_orientation_f1.gif
Creating animation for Flight 2 (3738 frames)...
Saved: animations/flight_log_11_20_25/flight_log_11_20_25_orientation_f2.gif
Creating animation for Flight 3 (2892 frames)...
Saved: animations/flight_log_11_20_25/flight_log_11_20_25_orientation_f2.gif
Creating animation for Flight 3 (2892 frames)...
Saved: animations/flight_log_11_20_25/flight_log_11_20_25_orientation_f3.gif

All animations saved to: animations/flight_log_11_20_25
Saved: animations/flight_log_11_20_25/flight_log_11_20_25_orientation_f3.gif

All animations saved to: animations/flight_log_11_20_25


## Summary

3D orientation animations have been generated and saved to the `animations/` directory. Each animation shows:
- A 3D rectangular prism representing the payload
- Real-time orientation based on yaw, pitch, and roll data
- Color-coded by flight state (GROUND, ASCENT, DESCENT, LANDED)
- Current time and orientation angles displayed on screen

**Note:** Animations are saved as GIF files. For smoother playback, consider reducing `skip_frames` parameter (will increase file size).